# Data ETL
*******************************
- transform and load Dataframes from 2 csv_files to tables in postgreSQL

In [1]:
# Depends
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import psqlpass

In [2]:
# pandas read csv
teamPerformData = pd.read_csv('../Resources/DataEtl/seasonAttendanceData.csv')
orgData = pd.read_csv('../Resources/scrape_collect/seasonPayrollData.csv')

In [3]:
# column transformation
teamPerformData.head()

,teamID,franchID,name,G,Ghome,yearID,Rank,W,L,park,attendance,HR,R,DivWin,WCWin,LgWin,WSWin,att_per_game
0,ATL,ATL,Atlanta Braves,162,81,1990,6,65,97,Atlanta-Fulton County Stadium,980129,162,682,N,0,N,N,12100.358025
1,BAL,BAL,Baltimore Orioles,161,80,1990,5,76,85,Memorial Stadium,2415189,132,669,N,0,N,N,30189.862500
2,BOS,BOS,Boston Red Sox,162,81,1990,1,88,74,Fenway Park II,2528986,106,699,Y,0,N,N,31222.049383
3,CAL,ANA,California Angels,162,81,1990,4,80,82,Anaheim Stadium,2555688,147,690,N,0,N,N,31551.703704
4,CHA,CHW,Chicago White Sox,162,80,1990,2,94,68,Comiskey Park,2002357,106,682,N,0,N,N,25029.462500


In [4]:
# change column names
print(teamPerformData.columns)

Index(['teamID', 'franchID', 'name', 'G', 'Ghome', 'yearID', 'Rank', 'W', 'L',
       'park', 'attendance', 'HR', 'R', 'DivWin', 'WCWin', 'LgWin', 'WSWin',
       'att_per_game'],
      dtype='object')


In [5]:
teamPerformData.rename(columns={
    'yearID': 'season',
    'teamID': 'team_abbrv',
    'franchID': 'franchise_abbrv',
    'name': 'team_name',
    'G': 'games_played',
    'Ghome': 'home_games_played',
    'Rank': 'season_rank',
    'W': 'wins',
    'L': 'losses',
    'park': 'home_park',
    'HR': 'home_runs',
    'R': 'runs_scored',
    'DivWin': 'division_winner',
    'WCWin': 'wild_card_winner',
    'LgWin': 'league_winner',
    'WSWin': 'world_series_winner',
}, inplace=True)

teamPerformData.columns

Index(['team_abbrv', 'franchise_abbrv', 'team_name', 'games_played',
       'home_games_played', 'season', 'season_rank', 'wins', 'losses',
       'home_park', 'attendance', 'home_runs', 'runs_scored',
       'division_winner', 'wild_card_winner', 'league_winner',
       'world_series_winner', 'att_per_game'],
      dtype='object')

In [6]:
# Checking UNIQUE VALUES FOR team name, team abbrv, season, franchise Abbrv
print('-------------------------------------------------------------------------------------------')
print(f'Unique Seasons: {(teamPerformData.season.unique())}')
print('-------------------------------------------------------------------------------------------')
print(f'Unique team names: {(teamPerformData.team_name.unique())}')
print('-------------------------------------------------------------------------------------------')
print(f'Unique team abbrvs: {(teamPerformData.team_abbrv.unique())}')
print('-------------------------------------------------------------------------------------------')
print(f'Unique franchise abbrvs: {(teamPerformData.franchise_abbrv.unique())}')
print('-------------------------------------------------------------------------------------------')


-------------------------------------------------------------------------------------------
Unique Seasons: [1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2021]
-------------------------------------------------------------------------------------------
Unique team names: ['Atlanta Braves' 'Baltimore Orioles' 'Boston Red Sox' 'California Angels'
 'Chicago White Sox' 'Chicago Cubs' 'Cincinnati Reds' 'Cleveland Indians'
 'Detroit Tigers' 'Houston Astros' 'Kansas City Royals'
 'Los Angeles Dodgers' 'Minnesota Twins' 'Milwaukee Brewers'
 'Montreal Expos' 'New York Yankees' 'New York Mets' 'Oakland Athletics'
 'Philadelphia Phillies' 'Pittsburgh Pirates' 'San Diego Padres'
 'Seattle Mariners' 'San Francisco Giants' 'St. Louis Cardinals'
 'Texas Rangers' 'Toronto Blue Jays' 'Colorado Rockies' 'Florida Marlins'
 'Anaheim Angels' 'Arizona Diamondbacks' 'Tampa Bay Devil Rays'
 'Los Angeles A

In [7]:
uniqueSeasons = teamPerformData.season.unique()
seasonsdf = pd.DataFrame(uniqueSeasons, columns=['season'])
seasonsdf['season'] = seasonsdf['season'].astype('string')
# to csv
seasonsdf.to_csv('output/seasons.csv', encoding='utf-8', index=False)

In [8]:
seasonsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   season  31 non-null     string
dtypes: string(1)
memory usage: 376.0 bytes


In [9]:
uniqueTeams = teamPerformData.team_name.unique()
teamDf = pd.DataFrame(uniqueTeams, columns=['team'])
# to csv
teamDf.to_csv('output/teams.csv', encoding='utf-8', index=False)

In [10]:
teamDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   team    35 non-null     object
dtypes: object(1)
memory usage: 408.0+ bytes


In [11]:
uniqueFranchise = teamPerformData.franchise_abbrv.unique()
franchiseDF = pd.DataFrame(uniqueFranchise, columns=['franchise'])
franchiseDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   franchise  30 non-null     object
dtypes: object(1)
memory usage: 368.0+ bytes


In [12]:
franchiseDF.to_csv('output/franchise.csv', encoding='utf-8', index=False)

In [13]:
uniqueAbbrv = teamPerformData.team_abbrv.unique()
teamAbbrvDF = pd.DataFrame(uniqueAbbrv, columns=['teamAbbrv'])
teamAbbrvDF.to_csv('output/teamabbrv.csv', encoding='utf-8', index=False)
teamAbbrvDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   teamAbbrv  35 non-null     object
dtypes: object(1)
memory usage: 408.0+ bytes


# Connecting to local Database
---
- After creating DataFrames for Unique Team and Season data:
 - We converted DataFrames to CSV FILES and imported into our TableSchemas in our Local PostgreSQL DB
 - Now Connecting to local DB to load other CSV converted DataFrames into our DataBase  


In [14]:
protocol = 'postgresql'
username = 'postgres'
password = psqlpass
host = 'localhost'
port = 5432
database_name = 'baseball_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [15]:
engine.table_names()

C:\Users\marko\AppData\Local\Temp\ipykernel_20952\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['season', 'teamabbrv', 'franchise', 'seasonattendance', 'team']

In [16]:
teamDat = pd.read_csv('output/teamdata.csv')
teamDat.head()

,team_name,batters_avg_age,pitchers_avg_age,Batting_Park_Factor,Pitching_Park_Factor,HoF,AllStars,AllTimeASapp,Est_Payroll,AvgGameStartTime,yearID
0,Atlanta Braves,27.4,25.8,105,106,2,1,13,"$15,065,501",2:48,1990
1,Baltimore Orioles,27.4,26.2,97,97,1,2,10,"$10,100,084",3:03,1990
2,Boston Red Sox,28.9,30.6,104,104,2,3,13,"$20,858,333",2:58,1990
3,California Angels,29.9,28.1,97,97,2,2,17,"$22,417,500",2:58,1990
4,Chicago Cubs,28.2,26.1,108,108,3,3,8,"$14,599,000",2:44,1990


In [19]:
teamDat.rename(columns={'team_name': 'team', 'yearID': 'season'}, inplace=True)
teamDat['season'] = teamDat['season'].astype('string')

In [20]:
team_payroll = teamDat[['season', 'team', 'Est_Payroll']].copy()
team_payroll

,season,team,Est_Payroll
0,1990,Atlanta Braves,"$15,065,501"
1,1990,Baltimore Orioles,"$10,100,084"
2,1990,Boston Red Sox,"$20,858,333"
3,1990,California Angels,"$22,417,500"
4,1990,Chicago Cubs,"$14,599,000"
...,...,...,...
903,2021,St. Louis Cardinals,"$155,329,667"
904,2021,Tampa Bay Rays,"$64,037,166"
905,2021,Texas Rangers,"$67,890,999"
906,2021,Toronto Blue Jays,"$121,694,071"


In [21]:
team_payroll.to_csv('output/team_payroll.csv', encoding='utf-8', index=True)

In [22]:
team_payroll.to_sql(name='team_payroll', con=engine, if_exists='append', index=False)

908